<a href="https://colab.research.google.com/github/JarekMaleszyk/data-science-project-sandbox/blob/main/MAGIC_Gamma_Telescope_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

Data from https://archive.ics.uci.edu/dataset/159/magic+gamma+telescope

In [131]:
N_NEIGHBORS = 5

In [132]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df = pd.read_csv("magic04.data", names=cols)
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [133]:
df['class'] = [1 if x == 'g' else 0 for x in df['class']]
df['class'] = df['class'].astype(int)
df.head()

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1


In [134]:
# for label in df.columns[:-1]:
#   plt.hist(df[df['class'] == 1][label], color='blue', label='gamma', alpha=0.7, density=True)
#   plt.hist(df[df['class'] == 0][label], color='red', label='hadron', alpha=0.7, density=True)
#   plt.title(label)
#   plt.ylabel('Probability')
#   plt.xlabel(label)
#   plt.legend()
#   plt.show()

## Train, validation, test dataset

In [135]:
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [136]:
def scale_dataset(dataframe, oversample=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
    ros = RandomOverSampler()
    X, y = ros.fit_resample(X, y)

  data = np.hstack((X, np.reshape(y, (-1, 1))))

  return data, X, y

In [137]:
print(len(train[train['class'] == 1]))
print(len(train[train['class'] == 0]))

7349
4063


In [138]:
train, X_train, y_train = scale_dataset(train, oversample=True)

In [139]:
print(len(y_train[y_train == 1]))
print(len(y_train[y_train == 0]))

7349
7349


In [140]:
valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
test, X_test, y_test = scale_dataset(test, oversample=False)

## K-Nearest Neighbors

In [141]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [142]:
knn_model = KNeighborsClassifier(n_neighbors=N_NEIGHBORS)
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [143]:
y_pred = knn_model.predict(X_test)

In [144]:
print(y_test)
print(y_pred)

[1 0 0 ... 1 0 0]
[0 0 1 ... 1 0 0]


In [145]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73      1319
           1       0.86      0.85      0.85      2485

    accuracy                           0.81      3804
   macro avg       0.79      0.79      0.79      3804
weighted avg       0.81      0.81      0.81      3804



## Naive Bayes

In [146]:
from sklearn.naive_bayes import GaussianNB

In [147]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB()

In [148]:
y_pred = nb_model.predict(X_test)

In [149]:
print(y_test)
print(y_pred)

[1 0 0 ... 1 0 0]
[1 1 1 ... 1 1 1]


In [150]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.40      0.50      1319
           1       0.74      0.89      0.81      2485

    accuracy                           0.72      3804
   macro avg       0.70      0.65      0.65      3804
weighted avg       0.71      0.72      0.70      3804



## Logistic Regression
sigmoid function

x0 - simple logistic regression

x0, x1, x2, ..., xn - multiply logistic regression

In [151]:
from sklearn.linear_model import LogisticRegression

In [152]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression()

In [153]:
y_pred = lr_model.predict(X_test)
print(y_test)
print(y_pred)

[1 0 0 ... 1 0 0]
[1 0 0 ... 1 0 0]


In [154]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.71      0.69      1319
           1       0.84      0.81      0.83      2485

    accuracy                           0.78      3804
   macro avg       0.76      0.76      0.76      3804
weighted avg       0.78      0.78      0.78      3804



## Support Vector Machines (SVM)

In [155]:
from sklearn.svm import SVC

In [156]:
svc_model = SVC()
svc_model.fit(X_train, y_train)

SVC()

In [157]:
y_pred = svc_model.predict(X_test)
print(y_test)
print(y_pred)

[1 0 0 ... 1 0 0]
[1 0 0 ... 1 0 0]


In [158]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.78      0.80      1319
           1       0.89      0.90      0.89      2485

    accuracy                           0.86      3804
   macro avg       0.85      0.84      0.85      3804
weighted avg       0.86      0.86      0.86      3804

